In [1]:
import json
import numpy as np
import pickle


# get 五言絕句
with open("./data/all_poetry.json",'r',encoding='utf-8') as f:
    json_content = json.load(f)
    
    poetries_content = []

    for poetry in json_content:
        poetry = ''.join(poetry['paragraphs'])
        
        # get 五言絕句
        if len(poetry) == 24:
            poetries_content.append(poetry)

In [2]:
#config

num_word = 26 # 字 + 標點符號 + start end
batch_size = 64


In [3]:
poetries_content[:10]

['繫纜石城下，恣吟懷暫開。江人橈艇子，將謂莫愁來。',
 '九十九岡遙，天寒雪未消。羸童牽瘦馬，不敢過危橋。',
 '駐馬高溪側，旅人千里情。雁山山下水，還作此泉聲。',
 '恨苦淚不落，耿然東北心。空囊與瘦馬，羈紲意應深。',
 '縱步不知遠，夕陽猶未回。好花隨處發，流水趂人來。',
 '閩越曾爲塞，將軍舊置營。我歌空感慨，西北望神京。',
 '自從別鑾殿，長門幾度春。不知金屋裏，更貯若爲人。',
 '踟躕下山婦，共申別離久。爲問織縑人，何必長相守。',
 '輪如明月盡，羅似薄雲穿。無由重掩笑，分在秋風前。',
 '萬事皆零落，平生不可思。惟餘酒中趣，不減少年時。']

In [4]:
len(poetries_content)

3925

In [5]:
# get all character
from collections import Counter



all_content = ''.join(poetries_content)
char_counter = Counter(all_content)

# all_char = set(all_content)

# len(all_char)

In [6]:
char_counter =  sorted(char_counter.items(), key=lambda x: -x[1])    #sort

In [7]:
char_counter

[('。', 7837),
 ('，', 7827),
 ('不', 1176),
 ('人', 876),
 ('風', 632),
 ('無', 608),
 ('一', 591),
 ('山', 565),
 ('日', 548),
 ('花', 537),
 ('來', 489),
 ('月', 477),
 ('何', 473),
 ('春', 470),
 ('水', 447),
 ('中', 436),
 ('上', 432),
 ('有', 418),
 ('時', 393),
 ('心', 387),
 ('知', 380),
 ('秋', 372),
 ('夜', 369),
 ('自', 350),
 ('見', 343),
 ('江', 336),
 ('君', 333),
 ('如', 327),
 ('相', 323),
 ('長', 318),
 ('年', 314),
 ('雲', 311),
 ('天', 308),
 ('爲', 294),
 ('去', 287),
 ('白', 286),
 ('生', 282),
 ('歸', 271),
 ('是', 267),
 ('處', 265),
 ('下', 264),
 ('明', 264),
 ('空', 256),
 ('多', 256),
 ('行', 253),
 ('未', 250),
 ('在', 243),
 ('寒', 240),
 ('青', 239),
 ('落', 237),
 ('今', 237),
 ('千', 234),
 ('得', 234),
 ('客', 227),
 ('此', 222),
 ('家', 221),
 ('草', 214),
 ('南', 210),
 ('飛', 207),
 ('聲', 206),
 ('獨', 200),
 ('清', 198),
 ('高', 196),
 ('莫', 194),
 ('誰', 194),
 ('子', 193),
 ('別', 193),
 ('金', 193),
 ('欲', 192),
 ('樹', 190),
 ('城', 189),
 ('還', 188),
 ('盡', 188),
 ('看', 188),
 ('路', 186),
 ('道', 185),
 ('事', 18

In [8]:
chars,_=zip(*char_counter)  
char2id_dict = dict(zip(chars, range(len(chars))))  

In [9]:
char2id_dict['髑']

3750

In [10]:
len(char2id_dict)

3752

In [11]:
START = '<start>'
END = "<EOS>"

char2id_dict[START] =len(char2id_dict)
char2id_dict[END] = len(char2id_dict)

In [12]:
char2id_dict[START]

3752

In [13]:
len(char2id_dict)

3754

In [14]:
char_size = len(char2id_dict)

In [15]:
id2char = {i:c for c,i in char2id_dict.items()}
id2char

{0: '。',
 1: '，',
 2: '不',
 3: '人',
 4: '風',
 5: '無',
 6: '一',
 7: '山',
 8: '日',
 9: '花',
 10: '來',
 11: '月',
 12: '何',
 13: '春',
 14: '水',
 15: '中',
 16: '上',
 17: '有',
 18: '時',
 19: '心',
 20: '知',
 21: '秋',
 22: '夜',
 23: '自',
 24: '見',
 25: '江',
 26: '君',
 27: '如',
 28: '相',
 29: '長',
 30: '年',
 31: '雲',
 32: '天',
 33: '爲',
 34: '去',
 35: '白',
 36: '生',
 37: '歸',
 38: '是',
 39: '處',
 40: '下',
 41: '明',
 42: '空',
 43: '多',
 44: '行',
 45: '未',
 46: '在',
 47: '寒',
 48: '青',
 49: '落',
 50: '今',
 51: '千',
 52: '得',
 53: '客',
 54: '此',
 55: '家',
 56: '草',
 57: '南',
 58: '飛',
 59: '聲',
 60: '獨',
 61: '清',
 62: '高',
 63: '莫',
 64: '誰',
 65: '子',
 66: '別',
 67: '金',
 68: '欲',
 69: '樹',
 70: '城',
 71: '還',
 72: '盡',
 73: '看',
 74: '路',
 75: '道',
 76: '事',
 77: '開',
 78: '將',
 79: '遠',
 80: '朝',
 81: '入',
 82: '玉',
 83: '雨',
 84: '門',
 85: '可',
 86: '色',
 87: '出',
 88: '葉',
 89: '萬',
 90: '向',
 91: '愁',
 92: '流',
 93: '三',
 94: '深',
 95: '似',
 96: '前',
 97: '東',
 98: '應',
 99: '裏',
 100: '猶',

In [16]:
# poetry2id_seqs

poetry2id_seqs = []

for poetry  in poetries_content:
    poetry2id_seqs.append([char2id_dict[START]]+[char2id_dict[c] for c in list(poetry) ]+[char2id_dict[END]])

In [17]:
poetry2id_seqs[0]

[3752,
 923,
 1537,
 129,
 70,
 40,
 1,
 1686,
 291,
 347,
 548,
 77,
 0,
 25,
 3,
 1236,
 1329,
 65,
 1,
 78,
 1004,
 63,
 91,
 10,
 0,
 3753]

In [18]:
content = "".join([id2char[c] for c in poetry2id_seqs[0]])
content

'<start>繫纜石城下，恣吟懷暫開。江人橈艇子，將謂莫愁來。<EOS>'

In [19]:
with open("./data/poetry2id_seqs.pickle",'wb') as f:
    pickle.dump(poetry2id_seqs,f)

In [20]:
with open("./data/char2id_dict.pickle",'wb') as f:
    pickle.dump(char2id_dict,f)

In [24]:
with open("./data/id2char_dict.pickle",'wb') as f:
    pickle.dump(id2char,f)

In [21]:
import torch


def make_one_hot_vec_target(c,c2id):
    return torch.LongTensor([c2id[c]])


one_hot_dict = {}

for c in char2id_dict:
    one_hot_dict.setdefault(c,make_one_hot_vec_target(c,char2id_dict))

In [22]:
one_hot_dict

{'。': tensor([0]),
 '，': tensor([1]),
 '不': tensor([2]),
 '人': tensor([3]),
 '風': tensor([4]),
 '無': tensor([5]),
 '一': tensor([6]),
 '山': tensor([7]),
 '日': tensor([8]),
 '花': tensor([9]),
 '來': tensor([10]),
 '月': tensor([11]),
 '何': tensor([12]),
 '春': tensor([13]),
 '水': tensor([14]),
 '中': tensor([15]),
 '上': tensor([16]),
 '有': tensor([17]),
 '時': tensor([18]),
 '心': tensor([19]),
 '知': tensor([20]),
 '秋': tensor([21]),
 '夜': tensor([22]),
 '自': tensor([23]),
 '見': tensor([24]),
 '江': tensor([25]),
 '君': tensor([26]),
 '如': tensor([27]),
 '相': tensor([28]),
 '長': tensor([29]),
 '年': tensor([30]),
 '雲': tensor([31]),
 '天': tensor([32]),
 '爲': tensor([33]),
 '去': tensor([34]),
 '白': tensor([35]),
 '生': tensor([36]),
 '歸': tensor([37]),
 '是': tensor([38]),
 '處': tensor([39]),
 '下': tensor([40]),
 '明': tensor([41]),
 '空': tensor([42]),
 '多': tensor([43]),
 '行': tensor([44]),
 '未': tensor([45]),
 '在': tensor([46]),
 '寒': tensor([47]),
 '青': tensor([48]),
 '落': tensor([49]),
 '今': tens

In [23]:
with open("./data/one_hot_dict.pickle",'wb') as f:
    pickle.dump(one_hot_dict,f)